In [1]:
#include <iostream>
#include <algorithm>
#include <utility>
#include <limits>
#include <vector>
#include <random>
#include <chrono>
#include <numeric>
#include <stdexcept>
#include <assert.h>
#include <math.h>

using namespace std;
using std::chrono::duration;

typedef pair<int, pair<unsigned int, unsigned int> > STNodeData;
typedef vector<STNodeData> STNodeVec;
typedef vector<int> IntVec;

struct Node {
    int val;
    Node* left = nullptr;
    Node* right = nullptr;
    pair<unsigned int, unsigned int> range;
};

Node* build_st_nodes(vector<int>& v);
int build_st_nodes_recurse(vector<int>& v, Node* curr_node, size_t i, size_t j);
int st_nodes_rmq(vector<int>& v, size_t i, size_t j, const Node* curr_node);

vector<STNodeData> *build_st_vec(vector<int>& v);
int build_st_vec_recurse(vector<int>& v, vector<STNodeData> &st_vec, size_t curr_idx, size_t i, size_t j);
int st_vec_rmq(vector<int>& v, size_t i, size_t j, const vector<STNodeData> &st_vec, size_t curr_idx);

vector<int>* build_st_small_vec(vector<int>& v);
int build_st_small_vec_recurse(vector<int>& v, vector<int>& st_vec, size_t curr_idx, size_t i, size_t j);
int st_small_vec_rmq(vector<int> &v, size_t i, size_t j, const vector<int>& st_vec, size_t curr_idx, size_t s, size_t e);

## Segment tree node pointer implementation

In [2]:
/*
 * Recurisvely builds the segment tree.
 */
int build_st_nodes_recurse(vector<int>& v, Node* curr_node, size_t i, size_t j) {
    curr_node->range.first = i;
    curr_node->range.second = j;
    if (i == j) {
        curr_node->val = v[i];
        return curr_node->val; // will this be (noticeably) faster than v[i] since curr_node->val is the latest thing accessed in memory?
    }
    size_t half = (j - i) / 2;
    curr_node->left = new Node;
    curr_node->right = new Node;
    int left_min = build_st_nodes_recurse(v, curr_node->left, i, i + half);
    int right_min = build_st_nodes_recurse(v, curr_node->right, ((j - i) % 2) ? (j - half) : (j - half + 1), j);
    curr_node->val = min(left_min, right_min);
    return curr_node->val;
}

In [3]:
/* Takes a vector<int> v and builds a segment tree out of it
 * for answering range minimum queries in O(log n) time. The
 * segment tree is represented with nodes and child pointers.
 *
 * Returns a pointer to the root of the constructed segment tree.
 */
Node* build_st_nodes(vector<int>& v) {
    if (v.empty())
        return nullptr;
    Node* root = new Node;
    build_st_nodes_recurse(v, root, 0, v.size() - 1);
    return root;
}

In [4]:
/* Takes a vector<int> v, two indices i, j representing
 * the beginning and end of the query range respectively,
 * and a pointer to the root of the constructed segment
 * tree.
 *
 * Returns the minimum element in v between i and j inclusive
 * using the segment tree data structure.
 */
int st_nodes_rmq(vector<int>& v, size_t i, size_t j, const Node* curr_node) {
    if (i > j || j >= v.size())
        throw invalid_argument("received invalid query range");
    if (i <= curr_node->range.first && j >= curr_node->range.second)
        return curr_node->val;
    if (j < curr_node->range.first || i > curr_node->range.second)
        return numeric_limits<int>::max();
    return min(st_nodes_rmq(v, i, j, curr_node->left), st_nodes_rmq(v, i, j, curr_node->right));
}

## Segment tree vector implementation

In [5]:
int build_st_vec_recurse(vector<int>& v, vector<STNodeData>& st_vec, size_t curr_idx, size_t i, size_t j) {
    // st_vec[curr_idx] is a pair<int, pair<unsigned int, unsigned int> >
    st_vec[curr_idx].second.first = i;
    st_vec[curr_idx].second.second = j;
    if (i == j) {
        st_vec[curr_idx].first = v[i];
        return st_vec[curr_idx].first;
    }
    size_t half = (j - i) / 2;
    int left_min = build_st_vec_recurse(v, st_vec, curr_idx * 2 + 1, i, i + half);
    int right_min = build_st_vec_recurse(v, st_vec, curr_idx * 2 + 2, ((j - i) % 2) ? (j - half) : (j - half + 1), j);
    st_vec[curr_idx].first = min(left_min, right_min);
    return st_vec[curr_idx].first;
}

In [6]:
STNodeVec* build_st_vec(vector<int>& v) {
    if (v.empty())
        return nullptr;
    vector<STNodeData>* st_vec = new vector<STNodeData>((int) 2 * pow(2.0, ceil(log2(v.size()))));
    build_st_vec_recurse(v, *st_vec, 0, 0, v.size() - 1);
    return st_vec;
}

In [7]:
int st_vec_rmq(vector<int> &v, size_t i, size_t j, const vector<STNodeData>& st_vec, size_t curr_idx) {
    if (i > j || j >= v.size())
        throw invalid_argument("received invalid query range");
    if (i <= st_vec[curr_idx].second.first && j >= st_vec[curr_idx].second.second)
        return st_vec[curr_idx].first;
    if (j < st_vec[curr_idx].second.first || i > st_vec[curr_idx].second.second)
        return numeric_limits<int>::max();
    return min(st_vec_rmq(v, i, j, st_vec, curr_idx * 2 + 1), st_vec_rmq(v, i, j, st_vec, curr_idx * 2 + 2));
}

## Segment tree vector implementation, no stored ranges

In [8]:
int build_st_small_vec_recurse(vector<int>& v, vector<int>& st_vec, size_t curr_idx, size_t i, size_t j) {
    if (i == j) {
        st_vec[curr_idx] = v[i];
        return st_vec[curr_idx];
    }
    size_t half = (j - i) / 2;
    int left_min = build_st_small_vec_recurse(v, st_vec, curr_idx * 2 + 1, i, i + half);
    int right_min = build_st_small_vec_recurse(v, st_vec, curr_idx * 2 + 2, ((j - i) % 2) ? (j - half) : (j - half + 1), j);
    st_vec[curr_idx] = min(left_min, right_min);
    return st_vec[curr_idx];
}

In [9]:
IntVec* build_st_small_vec(vector<int>& v) {
    if (v.empty())
        return nullptr;
    vector<int>* st_vec = new vector<int>((int) 2 * pow(2.0, ceil(log2(v.size()))));
    build_st_small_vec_recurse(v, *st_vec, 0, 0, v.size() - 1);
    return st_vec;
}

In [10]:
int st_small_vec_rmq(vector<int> &v, size_t i, size_t j, const vector<int>& st_vec, size_t curr_idx, size_t s, size_t e) {
    if (i > j || j >= v.size())
        throw invalid_argument("received invalid query range");
    if (i <= s && j >= e)
        return st_vec[curr_idx];
    if (j < s || i > e)
        return numeric_limits<int>::max();
    size_t half = (e - s) / 2;
    return min(st_small_vec_rmq(v, i, j, st_vec, curr_idx * 2 + 1, s, s + half),
               st_small_vec_rmq(v, i, j, st_vec, curr_idx * 2 + 2, ((e - s) % 2) ? (e - half) : (e - half + 1), e));
}

## A couple helper functions

In [11]:
int linear_rmq(vector<int>& v, size_t i, size_t j) {
    int ans = numeric_limits<int>::max();
    for (int k = i; k <= j; k++) {
        if (v[k] < ans)
            ans = v[k];
    }
    return ans;
}

In [12]:
/*
 * Destroys the tree rooted at the given curr_node. If given the pointer
 * to the root of the segment tree, it recursively deletes the entire tree.
 */
void destroy_st(Node* curr_node) {
    if (curr_node == nullptr)
        return;
    Node* left = curr_node->left;
    Node* right = curr_node->right;
    delete curr_node;
    destroy_st(left);
    destroy_st(right);
}

## Testing

In [13]:
random_device rd;
mt19937 rng(rd());
uniform_int_distribution<int> dist(-1000, 1000);

// preprocess and query times for nodes implementation
vector<double> n_preprocess_times;
vector<double> n_query_times;
// preprocess and query times for vec implementation
vector<double> v_preprocess_times;
vector<double> v_query_times;
// preprocess and query times for small vec implementation (no stored ranges)
vector<double> sv_preprocess_times;
vector<double> sv_query_times;

vector<int> n_vals;
int num_queries;

In [15]:
n_vals = {(int) 1e4, (int) 1e5, (int) 1e6, (int) 1e7, (int) 1e8};
num_queries = 50;

In [16]:
for (size_t i = 0; i < n_vals.size(); i++) {
    int n = n_vals[i]; // current size of v for timing
    vector<int> v(n);
    auto gen = [](){
        return dist(rng);
    };
    generate(v.begin(), v.end(), gen); // std::generate from <algorithm>; third param is a generator
    auto p_t1 = chrono::high_resolution_clock::now();
    Node* st = build_st_nodes(v); // pointer to root of constructed segment tree
    auto p_t2 = chrono::high_resolution_clock::now();
    n_preprocess_times.push_back(chrono::duration_cast<chrono::microseconds>(p_t2 - p_t1).count());
    vector<double> individual_query_times;
    for (int j = 0; j < num_queries; j++) {
        int s = rand() % n; // query range start
        int e = rand() % (n - s) + s; // query range end
        auto q_t1 = chrono::high_resolution_clock::now();
        int ans = st_nodes_rmq(v, s, e, st); // query answer (min element between indices s and e inclusive)
        auto q_t2 = chrono::high_resolution_clock::now();
        assert(ans == linear_rmq(v, s, e));
        individual_query_times.push_back(chrono::duration_cast<chrono::nanoseconds>(q_t2 - q_t1).count());
    }
    n_query_times.push_back(accumulate(individual_query_times.begin(), individual_query_times.end(), 0)
                         / num_queries);
    // free the memory allocated for the segment tree
    destroy_st(st);
}

In [17]:
for (int i = 0; i < n_vals.size(); i++) {
    cout << "Size: " << n_vals[i] << "\n";
    cout << "Preprocessing time in ms: " << n_preprocess_times[i] / 1000.0 << "\n";
    cout << "Avg query time in us (over " << num_queries << " queries): " << n_query_times[i] / 1000.0 << "\n\n";
}

Size: 10000
Preprocessing time in ms: 28.529
Avg query time in us (over 50 queries): 56.156

Size: 100000
Preprocessing time in ms: 281.234
Avg query time in us (over 50 queries): 72.803

Size: 1000000
Preprocessing time in ms: 2839.18
Avg query time in us (over 50 queries): 92.056

Size: 10000000
Preprocessing time in ms: 28354.6
Avg query time in us (over 50 queries): 110.536

Size: 100000000
Preprocessing time in ms: 273901
Avg query time in us (over 50 queries): 155.212



In [18]:
#pragma cling add_include_path("/Users/hashemelezabi/miniconda2/include")

#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"

In [19]:
#include <string>

xpl::figure n_plot;
xpl::linear_scale x_scale, y_scale;

xpl::lines n_p_line(x_scale, y_scale);
n_p_line.x = n_vals;
n_p_line.y = n_preprocess_times;
n_plot.add_mark(n_p_line);

vector<double> n_query_times_ms;
for (int i = 0; i < n_query_times.size(); i++) {
    n_query_times_ms.push_back(n_query_times[i] / 1000.0);
}

xpl::lines n_q_line(x_scale, y_scale);
n_q_line.x = n_vals;
n_q_line.y = n_query_times_ms;
n_plot.add_mark(n_q_line);

xpl::axis size_axis(x_scale), time_axis(y_scale);
time_axis.orientation = "vertical";
size_axis.label = "Size of input vector";
time_axis.label = "Time in milliseconds";

n_plot.add_axis(size_axis);
n_plot.add_axis(time_axis);

n_plot

A Jupyter widget